# HVM Small-Amplitude Oscillatory Shear (SAOS)

**Hybrid Vitrimer Model — Two Maxwell modes + permanent plateau**

## Learning Objectives

- Predict SAOS moduli for a 3-subnetwork vitrimer
- Identify G_P from the low-frequency plateau
- Observe the factor-of-2: τ_E_eff = 1/(2·k_BER_0)
- Temperature sweep showing Arrhenius shift of crossover

## Estimated Runtime

- ~30s (analytical)

## 1. Setup

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display

from rheojax.core.jax_config import safe_import_jax, verify_float64
from rheojax.models import HVMLocal

jax, jnp = safe_import_jax()
verify_float64()
import sys, os
sys.path.insert(0, os.path.dirname(os.path.abspath("")))
from utils.plotting_utils import (
    plot_nlsq_fit, display_arviz_diagnostics, plot_posterior_predictive
)

## 2. Theory

The HVM SAOS moduli are a superposition of two Maxwell modes plus a permanent plateau:

$$G'(\omega) = G_P + \frac{G_E \omega^2 \tau_{E,eff}^2}{1 + \omega^2 \tau_{E,eff}^2} + \frac{G_D \omega^2 \tau_D^2}{1 + \omega^2 \tau_D^2}$$

$$G''(\omega) = \frac{G_E \omega \tau_{E,eff}}{1 + \omega^2 \tau_{E,eff}^2} + \frac{G_D \omega \tau_D}{1 + \omega^2 \tau_D^2}$$

where $\tau_{E,eff} = 1/(2 k_{BER,0})$ (factor-of-2!) and $\tau_D = 1/k_d^D$.

**Key features:**
- **Low-ω:** $G'(\omega \to 0) = G_P$ (permanent network plateau)
- **High-ω:** $G'(\omega \to \infty) = G_P + G_E + G_D$ (total modulus)
- **Two loss peaks** in $G''$: one at $\omega \sim 1/\tau_{E,eff}$, one at $\omega \sim 1/\tau_D$

## 3. Model Setup

In [ ]:
# Create full HVM model
model = HVMLocal()
model.parameters.set_value("G_P", 5000.0)
model.parameters.set_value("G_E", 3000.0)
model.parameters.set_value("G_D", 1000.0)
model.parameters.set_value("nu_0", 1e10)
model.parameters.set_value("E_a", 80e3)
model.parameters.set_value("V_act", 1e-5)
model.parameters.set_value("T", 350.0)
model.parameters.set_value("k_d_D", 10.0)

# Check effective relaxation times
tau_E = model.get_vitrimer_relaxation_time()
tau_D = 1.0 / model.k_d_D
k_BER_0 = model.compute_ber_rate_at_equilibrium()
regime = model.classify_vitrimer_regime()

print(f"k_BER_0 = {k_BER_0:.4f} 1/s")
print(f"τ_E_eff = 1/(2·k_BER_0) = {tau_E:.4f} s")
print(f"τ_D     = 1/k_d_D      = {tau_D:.4f} s")
print(f"Regime: {regime}")
print(f"Limiting case: {model.get_limiting_case()}")
print(f"Network fractions: {model.get_network_fractions()}")

## 4. SAOS Prediction

In [ ]:
omega = np.logspace(-3, 3, 200)
G_prime, G_double_prime = model.predict_saos(omega)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# G', G'' vs omega
ax1.loglog(omega, G_prime, "k-", lw=2, label="G'")
ax1.loglog(omega, G_double_prime, "r-", lw=2, label="G''")
ax1.axhline(model.G_P, color="blue", ls=":", alpha=0.5, label=f"G_P = {model.G_P:.0f}")
ax1.axhline(model.G_P + model.G_E + model.G_D, color="green", ls=":", alpha=0.5,
            label=f"G_tot = {model.G_P + model.G_E + model.G_D:.0f}")
ax1.axvline(1/tau_E, color="gray", ls="--", alpha=0.4, label=f"1/τ_E_eff = {1/tau_E:.2f}")
ax1.axvline(1/tau_D, color="orange", ls="--", alpha=0.4, label=f"1/τ_D = {1/tau_D:.1f}")
ax1.set_xlabel("ω [rad/s]")
ax1.set_ylabel("G', G'' [Pa]")
ax1.set_title("HVM SAOS: Two Maxwell Modes + Plateau")
ax1.legend(fontsize=8)
ax1.grid(True, alpha=0.3, which="both")

# tan(delta)
tan_delta = G_double_prime / G_prime
ax2.semilogx(omega, tan_delta, "b-", lw=2)
ax2.set_xlabel("ω [rad/s]")
ax2.set_ylabel("tan(δ)")
ax2.set_title("Loss Tangent")
ax2.grid(True, alpha=0.3, which="both")

plt.tight_layout()
display(fig)
plt.close(fig)

## 5. Factor-of-2 Demonstration

In [ ]:
# Compare E-network contribution: Maxwell with τ_E = 1/k_BER_0 vs τ_E_eff = 1/(2·k_BER_0)
tau_E_naive = 1.0 / k_BER_0  # Without factor-of-2
tau_E_correct = tau_E          # With factor-of-2

G_E_prime_naive = model.G_E * (omega * tau_E_naive)**2 / (1 + (omega * tau_E_naive)**2)
G_E_prime_correct = model.G_E * (omega * tau_E_correct)**2 / (1 + (omega * tau_E_correct)**2)

fig, ax = plt.subplots(figsize=(8, 5))
ax.loglog(omega, G_prime - model.G_P, "k-", lw=2, label="HVM (full, minus G_P)")
ax.loglog(omega, G_E_prime_correct, "g--", lw=2, label=f"E-mode: τ_E_eff = {tau_E_correct:.3f}s (correct)")
ax.loglog(omega, G_E_prime_naive, "r:", lw=2, label=f"E-mode: τ_E = {tau_E_naive:.3f}s (wrong)")
ax.set_xlabel("ω [rad/s]")
ax.set_ylabel("G' - G_P [Pa]")
ax.set_title("Factor-of-2: τ_E_eff = 1/(2·k_BER_0), NOT 1/k_BER_0")
ax.legend()
ax.grid(True, alpha=0.3, which="both")
plt.tight_layout()
display(fig)
plt.close(fig)

## 6. Temperature Sweep (Arrhenius)

In [ ]:
temperatures = [300, 325, 350, 375, 400]
colors = plt.cm.RdYlBu(np.linspace(0.8, 0.2, len(temperatures)))

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

for T_val, color in zip(temperatures, colors):
    model.parameters.set_value("T", float(T_val))
    G_p, G_dp = model.predict_saos(omega)
    ax1.loglog(omega, G_p, "-", color=color, lw=1.5, label=f"T = {T_val} K")
    ax1.loglog(omega, G_dp, "--", color=color, lw=1, alpha=0.7)

ax1.set_xlabel("ω [rad/s]")
ax1.set_ylabel("G' (solid), G'' (dashed) [Pa]")
ax1.set_title("Temperature Sweep")
ax1.legend(fontsize=8)
ax1.grid(True, alpha=0.3, which="both")

# Arrhenius plot
model.parameters.set_value("T", 350.0)  # Reset
T_range = np.linspace(280, 450, 100)
inv_T, log_k = model.arrhenius_plot_data(T_range)
ax2.plot(inv_T, log_k, "b-", lw=2)
ax2.set_xlabel("1000/T [1/K]")
ax2.set_ylabel("log₁₀(k_BER_0) [1/s]")
ax2.set_title(f"Arrhenius Plot (E_a = {model.E_a/1e3:.0f} kJ/mol)")
ax2.grid(True, alpha=0.3)

plt.tight_layout()
display(fig)
plt.close(fig)

## 7. Limiting Cases Comparison

In [ ]:
# Compare limiting cases
partial = HVMLocal.partial_vitrimer(G_P=5000, G_E=3000, nu_0=1e10, E_a=80e3, V_act=1e-5, T=350.0)
zener = HVMLocal.zener(G_P=5000, G_D=1000, k_d_D=10.0)
pure_vit = HVMLocal.pure_vitrimer(G_E=3000, nu_0=1e10, E_a=80e3, V_act=1e-5, T=350.0)

fig, ax = plt.subplots(figsize=(8, 5))
for m, label, ls in [
    (model, "Full HVM", "-"),
    (partial, "Partial vitrimer (G_D=0)", "--"),
    (zener, "Zener (G_E=0)", "-."),
    (pure_vit, "Pure vitrimer (G_P=0, G_D=0)", ":"),
]:
    G_p, G_dp = m.predict_saos(omega)
    ax.loglog(omega, G_p, ls, lw=2, label=f"G' {label}")

ax.set_xlabel("ω [rad/s]")
ax.set_ylabel("G' [Pa]")
ax.set_title("Limiting Cases")
ax.legend(fontsize=8)
ax.grid(True, alpha=0.3, which="both")
plt.tight_layout()
display(fig)
plt.close(fig)

## Key Takeaways

1. **G'(ω→0) = G_P**: The permanent network provides a low-frequency plateau
2. **Factor-of-2**: τ_E_eff = 1/(2·k_BER_0) because both μ^E and μ^E_nat relax toward each other
3. **Arrhenius**: k_BER_0(T) = ν₀·exp(-E_a/RT) shifts crossover with temperature
4. **Limiting cases**: Setting subnetwork moduli to 0 recovers simpler models

## Next Steps

- **Notebook 02**: Stress relaxation (bi-exponential + plateau)
- **Notebook 03**: Startup shear (TST stress overshoot)